# Import Dataset from kaggle

In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("Cornell-University/arxiv")

print("Path to dataset files:", path)

Path to dataset files: /Users/leopard1566/.cache/kagglehub/datasets/Cornell-University/arxiv/versions/222


# Read into Pandas DataFrame

In [2]:
import pandas as pd
df = pd.read_json(path+'/arxiv-metadata-oai-snapshot.json', lines=True)
print(df.head())

          id           submitter  \
0  0704.0001      Pavel Nadolsky   
1  0704.0002        Louis Theran   
2  0704.0003         Hongjun Pan   
3  0704.0004        David Callan   
4  0704.0005  Alberto Torchinsky   

                                             authors  \
0  C. Bal\'azs, E. L. Berger, P. M. Nadolsky, C.-...   
1                    Ileana Streinu and Louis Theran   
2                                        Hongjun Pan   
3                                       David Callan   
4           Wael Abu-Shammala and Alberto Torchinsky   

                                               title  \
0  Calculation of prompt diphoton production cros...   
1           Sparsity-certifying Graph Decompositions   
2  The evolution of the Earth-Moon system based o...   
3  A determinant of Stirling cycle numbers counts...   
4  From dyadic $\Lambda_{\alpha}$ to $\Lambda_{\a...   

                                  comments  \
0  37 pages, 15 figures; published version   
1    To appear in

# Preprocess Data

In [3]:
#Filter out NAN category
df = df[df['categories'].notna()]
#Filter out withdrawn papers
df = df[df['abstract'].notna()]
df = df[~df['abstract'].str.contains('withdrawn', case=False, regex=False)]
print(df.head())

          id           submitter  \
0  0704.0001      Pavel Nadolsky   
1  0704.0002        Louis Theran   
2  0704.0003         Hongjun Pan   
3  0704.0004        David Callan   
4  0704.0005  Alberto Torchinsky   

                                             authors  \
0  C. Bal\'azs, E. L. Berger, P. M. Nadolsky, C.-...   
1                    Ileana Streinu and Louis Theran   
2                                        Hongjun Pan   
3                                       David Callan   
4           Wael Abu-Shammala and Alberto Torchinsky   

                                               title  \
0  Calculation of prompt diphoton production cros...   
1           Sparsity-certifying Graph Decompositions   
2  The evolution of the Earth-Moon system based o...   
3  A determinant of Stirling cycle numbers counts...   
4  From dyadic $\Lambda_{\alpha}$ to $\Lambda_{\a...   

                                  comments  \
0  37 pages, 15 figures; published version   
1    To appear in

In [4]:
df.shape

(2681257, 14)

# Import Vector Database and Model

In [5]:
from qdrant_client import models, QdrantClient
from sentence_transformers import SentenceTransformer

# Create vector database client and model client

In [6]:
model_encoder = SentenceTransformer('all-MiniLM-L6-v2')
qdrant_client = QdrantClient(":memory:")

Create collection to store 

In [7]:
qdrant_client.recreate_collection(
    collection_name = "arxiv",
    vectors_config = models.VectorParams(
        #Note that for our model, this vector size is 384
        size=model_encoder.get_sentence_embedding_dimension(),
        distance=models.Distance.COSINE
    )
)
if qdrant_client.collection_exists(collection_name="arxiv"):
    print("arxiv collection created successfully")

arxiv collection created successfully


/var/folders/w0/38fq1b_56633crmw8l8q9mgw0000gn/T/ipykernel_21332/3731503120.py:1: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  qdrant_client.recreate_collection(


# Insert quadrant data into collection
- Note to self: qdrant_client.upsert is used for bulk upload of points, while qdrant_client.insert is used for single point upload

- Before uploading the vectorized data, we load the dataframe into a key value pair dictionary to enable convenient iteration.

In [9]:
#Convert the WHOLE dataframe to a dictionary
#data_in_dict = df.to_dict(orient='records')

#Convert sample of the dataframe to a dictionary
sample_size=20000
data_in_dict = df.head(sample_size).to_dict(orient='records')

qdrant_client.upload_points(
    collection_name = "arxiv",
    points=[
        models.PointStruct(
            id = index,
            payload=doc,
            vector = model_encoder.encode(doc["abstract"]),
        )for index, doc in enumerate(data_in_dict)
    ]
)

# Query

In [10]:
user_prompt = "I want a research paper that focuses on Graphics"
limit_search_to = 10
#This will later be read from the user when the app is running on flask

query_prompt = "You are an AI agent searching for arXiv papers based on the following instructions: "+user_prompt

In [11]:
#Search based on user prompt
hits = qdrant_client.search(
    collection_name = "arxiv",
    query_vector=model_encoder.encode(query_prompt),
    limit=limit_search_to
)
#Print Queried Outputs
for hit in hits:
    print("TITLE: "+hit.payload["title"]+'\n\n', "arXivID: "+hit.payload["id"]+'\n\n', "AUTHORS: "+hit.payload["authors"]+'\n\n', "ABSTRACT: "+hit.payload["abstract"][0:100]+ '...' + '\n\n', '============break==========\n')


TITLE: On the realignment criterion and beyond

 arXivID: 0705.2328

 AUTHORS: Cosmo Lupo

 ABSTRACT:   The content of this paper is now available as part of arXiv:0802.2019
...

 ============break==========

TITLE: A solid angle polynomial with negative coefficients

 arXivID: 0707.1825

 AUTHORS: Steven V. Sam

 ABSTRACT:   This article has been replaced by arXiv:0906.4031
...

 ============break==========

TITLE: Secondary invariants for Frechet algebras, quasihomomorphisms, and the
  residue Chern character

 arXivID: 0706.1937

 AUTHORS: Denis Perrot

 ABSTRACT:   This paper has been replaced by arXiv:0804.1042 and arXiv:0804.1048
...

 ============break==========

TITLE: On the Energy-Momentum Problem in Static Einstein Universe

 arXivID: 0704.0525

 AUTHORS: Sezgin Aygun, Ismail Tarhan, Husnu Baysal

 ABSTRACT:   This paper has been removed by arXiv administrators because it plagiarizes
gr-qc/0410004, gr-qc/06...

 ============break==========

TITLE: Python for Education: Compu

/var/folders/w0/38fq1b_56633crmw8l8q9mgw0000gn/T/ipykernel_21332/122301538.py:2: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  hits = qdrant_client.search(


In [12]:
#TODO: Connect to local LLM using search result
search_results = [hit.payload for hit in hits]